### Related links:
- [AutoML Examples SDKv2](https://github.com/Azure/azureml-examples/tree/main/sdk/python/jobs/automl-standalone-jobs)
- [AutomL Examples  SDKv1](https://github.com/Azure/azureml-examples/tree/main/v1/python-sdk/tutorials/automl-with-azureml)
- [How to Configure AutoML Train v2](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train)


# AutoML: Train "the best" Regression model for the Hardware dataset.

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](../../../resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](../../../resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create an `AutoML regression Job` with the 'regression()' factory-function.
- Train the model using AmlCompute by submitting/running the AutoML regression training job
- Obtaing the model and score predictions with it

**Motivations** - This notebook explains how to setup and run an AutoML regression job. This is one of the nine ML-tasks supported by AutoML. Other ML-tasks are 'forecasting', 'classification', 'image classification', 'image object detection', 'nlp text classification', etc.

In this notebook, we go over how you can use AutoML for training a Regression model. We will use the Hardware Performance dataset to train and deploy the model to use in inference scenarios. The Regression goal is to predict the performance of certain combinations of hardware parts.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl
from azure.ai.ml import Input

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [2]:
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AzureML workspace
    subscription_id = "14585b9f-5c83-4a76-8055-42149123f99f"
    resource_group = "azureml_rg"
    workspace = "azureml_dev_jamg"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

We could not find config.json in: . or in its parent directories. 


### Show Azure ML Workspace information

In [3]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

subscription_id = ml_client.connections._subscription_id
resource_group = workspace.resource_group
workspace_name = ml_client.workspace_name

output = {}
output["Workspace"] = workspace_name
output["Subscription ID"] = subscription_id
output["Resource Group"] = resource_group
output["Location"] = workspace.location
output

{'Workspace': 'azureml_dev_jamg',
 'Subscription ID': '14585b9f-5c83-4a76-8055-42149123f99f',
 'Resource Group': 'azureml_rg',
 'Location': 'eastus'}

# 2. MLTable with input Training Data

## 2.1. Create MLTable data input
Please make use of the MLTable files present within the data folder at the same location (in the repo) as this notebook.

In [4]:
# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data/training-mltable-folder"
)

# WITH REMOTE PATH If available already in the cloud/workspace-blob-store
# my_training_data_input  = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/my-regression-mltable")

# 3. Compute target setup

### Create or Attach existing AmlCompute
A compute target is required to execute the Automated ML run. In this tutorial, you create AmlCompute as your training compute resource.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

#### Creation of AmlCompute takes approximately 5 minutes. 
If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [5]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "cpu-cluster"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="STANDARD_DS12_V2",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=6,
    )
    ml_client.begin_create_or_update(compute_config).result()

Found existing compute target.


# 4. Configure and run the AutoML Regression training job
In this section we will configure and run the AutoML regression job.

## 4.1 Configure the job through the regression() factory function

### regression() function parameters:

The `regression()` factory function allows user to configure AutoML for the regression task for the most common scenarios with the following properties.

- `target_column_name` - The name of the column to target for predictions. It must always be specified. This parameter is applicable to 'training_data', 'validation_data' and 'test_data'.
- `primary_metric` - The metric that AutoML will optimize for model selection.
- `training_data` - The data to be used for training. It should contain both training feature columns and a target column. Optionally, this data can be split for segregating a validation or test dataset. 
You can use a registered MLTable in the workspace using the format '<mltable_name>:<version>' OR you can use a local file or folder as a MLTable. For e.g Input(mltable='my_mltable:1') OR Input(mltable=MLTable(local_path="./data"))
The parameter 'training_data' must always be provided.
- `compute` - The compute on which the AutoML job will run. In this example we are using a compute called 'cpu-cluster' present in the workspace. You can replace it any other compute in the workspace. 
- `name` - The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
- `experiment_name` - The name of the Experiment. An Experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment.

### set_limits() function parameters:
This is an optional configuration method to configure limits parameters such as timeouts.     
    
- `timeout_minutes` - Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. This timeout includes setup, featurization and training runs but does not include the ensembling and model explainability runs at the end of the process since those actions need to happen once all the trials (children jobs) are done. If not specified, the default job's total timeout is 6 days (8,640 minutes). To specify a timeout less than or equal to 1 hour (60 minutes), make sure your dataset's size is not greater than 10,000,000 (rows times column) or an error results.

- `trial_timeout_minutes` - Maximum time in minutes that each trial (child job) can run for before it terminates. If not specified, a value of 1 month or 43200 minutes is used.
    
- `max_trials` - The maximum number of trials/runs each with a different combination of algorithm and hyperparameters to try during an AutoML job. If not specified, the default is 1000 trials. If using 'enable_early_termination' the number of trials used can be smaller.
    
- `max_concurrent_trials` - Represents the maximum number of trials (children jobs) that would be executed in parallel. It's a good practice to match this number with the number of nodes your cluster.
    
- `enable_early_termination` - Whether to enable early termination if the score is not improving in the short term. 
    

In [6]:
# general job parameters
max_trials = 5
exp_name = "dpv2-regression-experiment"

In [7]:
# Create the AutoML regression job with the related factory-function.

regression_job = automl.regression(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name="ERP",
    primary_metric="R2Score",
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"my_custom_tag": "My custom value"},
)

# Limits are all optional
regression_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=max_trials,
    # max_concurrent_trials = 4,
    # max_cores_per_trial: -1,
    enable_early_termination=True,
)

Class RegressionJob: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [8]:
# Configure custom featurization
# You can skip this cell to run AutoML using automatic featurization
from azure.ai.ml.automl import ColumnTransformer

transformer_params = {
    "imputer": [
        ColumnTransformer(fields=["CACH"], parameters={"strategy": "most_frequent"}),
        ColumnTransformer(fields=["PRP"], parameters={"strategy": "most_frequent"}),
    ],
}
regression_job.set_featurization(
    mode="custom",
    transformer_params=transformer_params,
    blocked_transformers=["LabelEncoder"],
    column_name_and_types={"CHMIN": "Categorical"},
)

## 4.2 Run the Command
Using the `MLClient` created earlier, we will now run this Command in the workspace.

In [9]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    regression_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

Created job: RegressionJob({'log_verbosity': <LogVerbosity.INFO: 'Info'>, 'task_type': <TaskType.REGRESSION: 'Regression'>, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'jolly_onion_tm659cl2bv', 'description': None, 'tags': {'my_custom_tag': 'My custom value'}, 'properties': {}, 'id': '/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/resourceGroups/azureml_rg/providers/Microsoft.MachineLearningServices/workspaces/azureml_dev_jamg/jobs/jolly_onion_tm659cl2bv', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.SystemData object at 0x7feb303aa020>, 'serialize': <msrest.serialization.Serializer object at 0x7feb30dd8fa0>, 'inputs': {}, 'display_name': 'jolly_onion_tm659cl2bv', 'experiment_name': 'dpv2-regression-experiment', 'compute': 'cpu-cluster', 'services': {'Tracking': <azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.JobSe

### Wait until the AutoML job is finished
ml_client.jobs.stream(returned_job.name) waits until the specified job is finished

In [10]:
# Wait for job to complete and stream updates
ml_client.jobs.stream(returned_job.name)

RunId: jolly_onion_tm659cl2bv
Web View: https://ml.azure.com/runs/jolly_onion_tm659cl2bv?wsid=/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/resourcegroups/azureml_rg/workspaces/azureml_dev_jamg

Execution Summary
RunId: jolly_onion_tm659cl2bv
Web View: https://ml.azure.com/runs/jolly_onion_tm659cl2bv?wsid=/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/resourcegroups/azureml_rg/workspaces/azureml_dev_jamg



In [11]:
# Get a URL for the status of the job
returned_job.services["Studio"].endpoint

'https://ml.azure.com/runs/jolly_onion_tm659cl2bv?wsid=/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/resourcegroups/azureml_rg/workspaces/azureml_dev_jamg&tid=16b3c013-d300-468d-ac64-7eda0820b6d3'

In [14]:
print(returned_job.name)

jolly_onion_tm659cl2bv


# 5. Retrieve the Best Trial (Best Model's trial/run)
Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

## Initialize MLFlow Client
The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface. 
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

*IMPORTANT*, you need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow

    pip install mlflow

### Obtain the tracking URI for MLFlow

In [4]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/resourceGroups/azureml_rg/providers/Microsoft.MachineLearningServices/workspaces/azureml_dev_jamg


In [5]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/resourceGroups/azureml_rg/providers/Microsoft.MachineLearningServices/workspaces/azureml_dev_jamg


In [6]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

### Get the AutoML parent Job

In [8]:
job_name = returned_job.name
job_name = 'jolly_onion_tm659cl2bv'

# Example if providing an specific Job name/ID
# job_name = "b4e95546-0aa1-448e-9ad6-002e3207b4fc"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'explained_variance': 0.8934360381091077,
 'mean_absolute_error': 13.545632733661114,
 'mean_absolute_percentage_error': 8.756661570034687,
 'median_absolute_error': 2.46805419921875,
 'normalized_mean_absolute_error': 0.011075742218856186,
 'normalized_median_absolute_error': 0.0020180328693530257,
 'normalized_root_mean_squared_error': 0.03397167276275391,
 'normalized_root_mean_squared_log_error': 0.032270099470456816,
 'r2_score': 0.8890376716187035,
 'root_mean_squared_error': 41.54735578884804,
 'root_mean_squared_log_error': 0.1403578669573778,
 'spearman_correlation': 0.9825951205888839}, params={}, tags={'_azureml.ComputeTargetType': 'STANDARD_DS11_V2',
 'automl_best_child_run_id': 'jolly_onion_tm659cl2bv_1',
 'fit_time': '',
 'iteration': '',
 'mlflow.rootRunId': 'jolly_onion_tm659cl2bv',
 'mlflow.runName': 'jolly_onion_tm659cl2bv',
 'mlflow.user': 'Jose Medina Gomez',
 'model_explain_best_run_child_id': 'jolly_onion_tm659cl2bv_1',
 

In [9]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'my_custom_tag': 'My custom value', 'model_explain_run': 'best_run', '_azureml.ComputeTargetType': 'STANDARD_DS11_V2', 'pipeline_id_000': 'faf12f74cf9bbd358ca5525682c5030d36f7be7c;4bc4ec47eb8df2d5d68b361cd60120e65196f757;5cc37daec73ea64276ef956449645cdb519fdfc6;__AutoML_Ensemble__;__AutoML_Stack_Ensemble__', 'score': '', 'predicted_cost': '', 'fit_time': '', 'training_percent': '', 'iteration': '', 'run_preprocessor': '', 'run_algorithm': '', 'automl_best_child_run_id': 'jolly_onion_tm659cl2bv_1', 'model_explain_best_run_child_id': 'jolly_onion_tm659cl2bv_1', 'mlflow.rootRunId': 'jolly_onion_tm659cl2bv', 'mlflow.runName': 'jolly_onion_tm659cl2bv', 'mlflow.user': 'Jose Medina Gomez'}


## Get the AutoML best child run

In [10]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id:  jolly_onion_tm659cl2bv_1
Best child run: 
<Run: data=<RunData: metrics={'explained_variance': 0.8934360381091077,
 'mean_absolute_error': 13.545632733661114,
 'mean_absolute_percentage_error': 8.756661570034687,
 'median_absolute_error': 2.46805419921875,
 'normalized_mean_absolute_error': 0.011075742218856186,
 'normalized_median_absolute_error': 0.0020180328693530257,
 'normalized_root_mean_squared_error': 0.03397167276275391,
 'normalized_root_mean_squared_log_error': 0.032270099470456816,
 'r2_score': 0.8890376716187035,
 'root_mean_squared_error': 41.54735578884804,
 'root_mean_squared_log_error': 0.1403578669573778,
 'spearman_correlation': 0.9825951205888839}, params={}, tags={'mlflow.parentRunId': 'jolly_onion_tm659cl2bv',
 'mlflow.rootRunId': 'jolly_onion_tm659cl2bv',
 'mlflow.runName': 'khaki_rocket_zpkbsfyb',
 'mlflow.user': 'Jose Medina Gomez',
 'model_explain_run_id': 'jolly_onion_tm659cl2bv_ModelExplain',
 'model_explanation': 'True'}>, info=<Run

## Get best model run's metrics

Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [11]:
best_run.data.metrics

{'mean_absolute_percentage_error': 8.756661570034687,
 'normalized_root_mean_squared_log_error': 0.032270099470456816,
 'normalized_root_mean_squared_error': 0.03397167276275391,
 'mean_absolute_error': 13.545632733661114,
 'root_mean_squared_log_error': 0.1403578669573778,
 'normalized_mean_absolute_error': 0.011075742218856186,
 'explained_variance': 0.8934360381091077,
 'r2_score': 0.8890376716187035,
 'root_mean_squared_error': 41.54735578884804,
 'spearman_correlation': 0.9825951205888839,
 'normalized_median_absolute_error': 0.0020180328693530257,
 'median_absolute_error': 2.46805419921875}

## Download the best model locally

Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [12]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [13]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

/tmp/ipykernel_25018/2387391148.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  local_path = mlflow_client.download_artifacts(


Artifacts downloaded in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/jomedincpu/code/Users/jomedin/taxi-fare-predictions/notebooks/AutoML/artifact_downloads/outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'generated_code', 'internal_cross_validated_models.pkl', 'mlflow-model', 'model.pkl', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


In [14]:
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

['conda.yaml', 'MLmodel', 'model.pkl', 'python_env.yaml', 'requirements.txt']

# 6. Register Best Model and Deploy

## 6.1 Create managed online endpoint

In [15]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azure.ai.ml.constants import ModelType

In [16]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "regression-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
    tags={"foo": "bar"},
)

In [18]:
ml_client.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://regression-02090312349920.eastus.inference.ml.azure.com/score', 'swagger_uri': 'https://regression-02090312349920.eastus.inference.ml.azure.com/swagger.json', 'name': 'regression-02090312349920', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/resourcegroups/azureml_rg/providers/microsoft.machinelearningservices/workspaces/azureml_dev_jamg/onlineendpoints/regression-02090312349920', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:607be527-f51c-44df-a376-cb3876e5ac0b:f84d6e04-943c-495e-a01d-b179b71dd7fc?api-version=2022-02-01-preview'}, 'id': '/subscriptions/14585b9f-5c83-4a76-8055-42149123f99

## 6.2 Register best model and deploy

### Register model

In [19]:
model_name = "hardware-performance-model"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="my sample regression model",
    type=AssetTypes.MLFLOW_MODEL,
)

# for downloaded file
# model = Model(path="artifact_downloads/outputs/model.pkl", name=model_name)

registered_model = ml_client.models.create_or_update(model)

In [20]:
registered_model.id

'/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/resourceGroups/azureml_rg/providers/Microsoft.MachineLearningServices/workspaces/azureml_dev_jamg/models/hardware-performance-model/versions/1'

### Deploy

In [21]:
deployment = ManagedOnlineDeployment(
    name="hardware-performance-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS3_V2",
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [22]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint regression-02090312349920 exists
Creating/updating online deployment hardware-performance-deploy 

...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Done (40m 50s)


AttributeError: 'NoneType' object has no attribute 'result'

In [23]:
# hardware performance deployment to take 100% traffic
endpoint.traffic = {"hardware-performance-deploy": 100}
ml_client.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://regression-02090312349920.eastus.inference.ml.azure.com/score', 'swagger_uri': 'https://regression-02090312349920.eastus.inference.ml.azure.com/swagger.json', 'name': 'regression-02090312349920', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/resourcegroups/azureml_rg/providers/microsoft.machinelearningservices/workspaces/azureml_dev_jamg/onlineendpoints/regression-02090312349920', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/14585b9f-5c83-4a76-8055-42149123f99f/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:607be527-f51c-44df-a376-cb3876e5ac0b:9703dcfc-2d85-4a6a-a3a9-680620282b39?api-version=2022-02-01-preview'}, 'id': '/subscriptions/14585b9f-5c83-4a76-8055-42149123f99

### Test the deployment

In [24]:
# test the blue deployment with some sample data
import pandas as pd

test_data = pd.read_csv("./data/training-mltable-folder/training-machine-dataset.csv")

test_data = test_data.drop("ERP", axis=1)

test_data_json = test_data.to_json(orient="records", indent=4)
data = (
    '{ \
          "input_data": {"data": '
    + test_data_json
    + "}}"
)

request_file_name = "sample-request-hardware-performance.json"

with open(request_file_name, "w") as request_file:
    request_file.write(data)

# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="hardware-performance-deploy",
    request_file="sample-request-hardware-performance.json",
)

'[198.99435424804688, 253.0391082763672, 252.9599151611328, 253.0175018310547, 131.99099731445312, 289.96868896484375, 380.9918212890625, 381.06829833984375, 748.9801635742188, 1238.000244140625, 22.97747230529785, 24.01284408569336, 69.98802947998047, 117.01985168457031, 14.991435050964355, 63.99908447265625, 22.99995994567871, 29.000783920288086, 22.00456428527832, 123.99982452392578, 34.98915100097656, 39.04862594604492, 40.00320816040039, 44.99788284301758, 27.99527359008789, 21.00965118408203, 28.039894104003906, 21.998796463012695, 27.985389709472656, 26.968013763427734, 102.00531005859375, 101.98802185058594, 74.0141372680664, 73.98561096191406, 137.97744750976562, 136.01246643066406, 23.004512786865234, 29.030193328857422, 44.00531005859375, 30.02463150024414, 40.920631408691406, 74.0069351196289, 74.00098419189453, 73.96170043945312, 54.094444274902344, 40.90602493286133, 18.004688262939453, 27.978290557861328, 36.00728988647461, 37.99821472167969, 33.985595703125, 19.01552009

In [28]:
print(data)

{           "input_data": {"data": [
    {
        "VendorName":"adviser",
        "ModelName":"32\/60",
        "MYCT":125,
        "MMIN":256,
        "MMAX":6000,
        "CACH":256,
        "CHMIN":16,
        "CHMAX":128,
        "PRP":198
    },
    {
        "VendorName":"amdahl",
        "ModelName":"470v\/7",
        "MYCT":29,
        "MMIN":8000,
        "MMAX":32000,
        "CACH":32,
        "CHMIN":8,
        "CHMAX":32,
        "PRP":269
    },
    {
        "VendorName":"amdahl",
        "ModelName":"470v\/7a",
        "MYCT":29,
        "MMIN":8000,
        "MMAX":32000,
        "CACH":32,
        "CHMIN":8,
        "CHMAX":32,
        "PRP":220
    },
    {
        "VendorName":"amdahl",
        "ModelName":"470v\/7b",
        "MYCT":29,
        "MMIN":8000,
        "MMAX":32000,
        "CACH":32,
        "CHMIN":8,
        "CHMAX":32,
        "PRP":172
    },
    {
        "VendorName":"amdahl",
        "ModelName":"470v\/7c",
        "MYCT":29,
        "MMIN":8000

In [25]:
# wait and delete endpoint
import time

time.sleep(60)

### Get endpoint details

In [26]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'hardware-performance-deploy': 100}
https://regression-02090312349920.eastus.inference.ml.azure.com/score


### Delete the deployment and endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

# Next Step: Load the best model and try predictions

Loading the models locally assume that you are running the notebook in an environment compatible with the model. The list of dependencies that is expected by the model is specified in the MLFlow model produced by AutoML (in the 'conda.yaml' file within the mlflow-model folder).

Since the AutoML model was trained remotelly in a different environment with different dependencies to your current local conda environment where you are running this notebook, if you want to load the model you have several options:

1. A recommended way to locally load the model in memory and try predictions is to create a new/clean conda environment with the dependencies specified in the conda.yml file within the MLFlow model's folder, then use MLFlow to load the model and call .predict() as explained in the notebook **mlflow-model-local-inference-test.ipynb** in this same folder.

2. You can install all the packages/dependencies specified in conda.yml into your current conda environment you used for using Azure ML SDK and AutoML. MLflow SDK also have a method to install the dependencies in the current environment. However, this option could have risks of package version conflicts depending on what's installed in your current environment.

3. You can also use: mlflow models serve -m 'xxxxxxx'

# Next Steps
You can see further examples of other AutoML tasks such as Image-Classification, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.